In [1]:
%matplotlib inline
%config InlineBackend.figure_format = "retina"

In [2]:
# Author: xiebo

# Python implementation: CPython
# Python version       : 3.8.5
# IPython version      : 8.12.3

# torch      : 2.2.0+cu118
# torchdata  : 0.7.1
# torchtext  : 0.17.0
# torchvision: 0.17.0+cu118
# torchaudio : 2.2.0+cu118
# openai     : 1.10.0
# langchain  : 0.1.5
# tiktoken   : 0.5.2
# numpy      : 1.24.4
# pandas     : 2.0.3
# sklearn    : 1.3.2
# matplotlib : 3.7.4
# scipy      : 1.10.1

%load_ext watermark
%watermark -a 'xiebo' -d -t -v -p torch,torchdata,torchtext,torchvision,torchaudio,openai,langchain,tiktoken,numpy,pandas,sklearn,matplotlib,scipy

Author: xiebo

Python implementation: CPython
Python version       : 3.8.5
IPython version      : 8.12.3

torch      : 2.2.0+cu118
torchdata  : 0.7.1
torchtext  : 0.17.0
torchvision: 0.17.0+cu118
torchaudio : 2.2.0+cu118
openai     : 1.10.0
langchain  : 0.1.5
tiktoken   : 0.5.2
numpy      : 1.24.4
pandas     : 2.0.3
sklearn    : 1.3.2
matplotlib : 3.7.4
scipy      : 1.10.1



In [3]:
import sys
sys.path.append("..")
sys.path.append("../..")

from project_utils import *

In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
set_jupyter_cell_width(45)

In [6]:
fix_all_seed()

In [7]:
print_closeai()

OPENAI_BASE_URL: https://api.closeai-proxy.xyz/v1
OPENAI_API_KEY: 8V55bZBNNrS0keILgUUIQwW1RWO-Ys4siyk9zwk2i27DwqoCzaQ


## 简单的 promp

In [8]:
#### 定义任务描述和输入
# 测试LLM
# 任务描述
instruction = """
你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称，月费价格，月流量。
根据用户输入，识别用户在上述三种属性上的倾向。
"""

# 用户输入
input_text = """
办个100G的套餐。
"""

# prompt 模版。instruction 和 input_text 会被替换为上面的内容
prompt = f"""
{instruction}

用户输入：
{input_text}
"""

# prompt_token_count = 94, completion_token_count = 77, total_token_count = 171.
# choice_count = 1, cost time = 2.4s, finish_reason = stop.
# 用户在流量套餐产品的选择条件上的倾向为：
# - 名称：用户倾向选择100G的套餐。
# - 月费价格：用户未提及对月费价格的倾向。
# - 月流量：用户倾向选择100G的套餐。
content = get_chat_completion_content(prompt)
print(content)

用户在流量套餐产品的选择条件上的倾向为：
- 名称：用户倾向选择100G的套餐。
- 月费价格：用户未提及对月费价格的倾向。
- 月流量：用户倾向选择100G的套餐。


## 增加对于输出格式的要求

In [9]:
# 从上面的结果可以看出 LLM 理解了我们的需求。
# 但是自然语言的输出，不方便我们形成最终的决策。

# 因此我们需要对输出的格式进行约定，形成格式化的输出，**推荐使用 JSON 格式输出**

# 输出格式限定
output_format = """
以 JSON 格式输出
"""

# 加入输出格式
prompt = f"""
{instruction}

{output_format}

用户输入：
{input_text}
"""

# {
#   "名称": "100G套餐",
#   "月费价格": "未知",
#   "月流量": "100G"
# }
content = get_chat_completion_content(prompt)
print(content)

{
  "名称": "100G套餐",
  "月费价格": "未提及",
  "月流量": "100G"
}


## 增加对 json 格式的细节描述，GPT3.5、4等大模型是经过code资料训练过的，所以对 JSON 格式的信息是有较高敏感度的。

In [10]:
# 任务描述增加了字段的**英文标识符**
instruction = """
你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称(name)，月费价格(price)，月流量(data)。
根据用户输入，识别用户在上述三种属性上的倾向。
"""

# 输出格式增加了各种定义、约束
output_format = """
以JSON格式输出。
1. name字段的取值为string类型，取值必须为以下之一：经济套餐、畅游套餐、无限套餐、校园套餐 或 null；

2. price字段的取值为一个结构体 或 null，包含两个字段：
(1) operator, string类型，取值范围：'<='（小于等于）, '>=' (大于等于), '=='（等于）
(2) value, int类型

3. data字段的取值为取值为一个结构体 或 null，包含两个字段：
(1) operator, string类型，取值范围：'<='（小于等于）, '>=' (大于等于), '=='（等于）
(2) value, int类型或string类型，string类型只能是'无上限'

4. 用户的意图可以包含按price或data排序，以sort字段标识，取值为一个结构体：
(1) 结构体中以"ordering"="descend"表示按降序排序，以"value"字段存储待排序的字段
(2) 结构体中以"ordering"="ascend"表示按升序排序，以"value"字段存储待排序的字段

只输出中只包含用户提及的字段，不要猜测任何用户未直接提及的字段，不输出值为null的字段。
"""

# {
#   "data": {
#     "operator": ">=",
#     "value": 100
#   }
# }
# input_text = "办个100G以上的套餐"

# {
#   "name": "无限套餐"
# }
# input_text = "我要无限量套餐"

# {
#   "name": "经济套餐",
#   "price": {
#     "operator": "<=",
#     "value": 50
#   }
# }
# input_text = "有没有便宜的套餐"

# {
#   "price": {
#     "operator": "<=",
#     "value": 100
#   },
#   "data": {
#     "operator": ">=",
#     "value": 10
#   },
#   "sort": {
#     "ordering": "descend",
#     "value": "data"
#   }
# }
# input_text = "有没有便宜的套餐，但流量能多一些"

# {
#   "data": {
#     "operator": ">=",
#     "value": "无上限"
#   }
# }
input_text = "我只要流量足够多的套餐"

prompt = f"""
{instruction}

{output_format}

用户输入：
{input_text}
"""

content = get_chat_completion_content(prompt)
print(content)

{
  "data": {
    "operator": ">=",
    "value": "无上限"
  }
}


## 增加 few shot

In [11]:
examples = """
便宜的套餐：{"sort":{"ordering"="ascend","value"="price"}}
有没有不限流量的：{"data":{"operator":"==","value":"无上限"}}
流量大的：{"sort":{"ordering"="descend","value"="data"}}
100G以上流量的套餐最便宜的是哪个：{"sort":{"ordering"="ascend","value"="price"},"data":{"operator":">=","value":100}}
月费不超过200的：{"price":{"operator":"<=","value":200}}
就要月费180那个套餐：{"price":{"operator":"==","value":180}}
经济套餐：{"name":"经济套餐"}
"""

# {"sort":{"ordering":"ascend","value":"price"}}
# input_text = "有没有便宜的套餐"

# {"name":"无限套餐"}
# input_text = "有没有土豪套餐"

# {"data":{"operator":">=","value":200},"sort":{"ordering":"ascend","value":"price"}}
# input_text = "办个200G的套餐"

# {"sort":{"ordering":"descend","value":"data"}}
# input_text = "有没有流量大的套餐"

# {"price":{"operator":"<=","value":200},"sort":{"ordering":"descend","value":"data"}}
# input_text = "200元以下，流量大的套餐有啥"

# {"data":{"operator":"==","value":10}}
input_text = "你说那个10G的套餐，叫啥名字"

# 有了例子
prompt = f"""
{instruction}

{output_format}

例如：
{examples}

用户输入：
{input_text}

"""

content = get_chat_completion_content(prompt)
print(content)

{"data":{"operator":"==","value":10}}


## 在 prompt 中增加多轮对话的例子

In [12]:
instruction = """
你的任务是识别用户对手机流量套餐产品的选择条件。
每种流量套餐产品包含三个属性：名称(name)，月费价格(price)，月流量(data)。
根据对话上下文，识别用户在上述属性上的倾向。识别结果要包含整个对话的信息。
"""

# 输出描述
output_format = """
以JSON格式输出。
1. name字段的取值为string类型，取值必须为以下之一：经济套餐、畅游套餐、无限套餐、校园套餐 或 null；

2. price字段的取值为一个结构体 或 null，包含两个字段：
(1) operator, string类型，取值范围：'<='（小于等于）, '>=' (大于等于), '=='（等于）
(2) value, int类型

3. data字段的取值为取值为一个结构体 或 null，包含两个字段：
(1) operator, string类型，取值范围：'<='（小于等于）, '>=' (大于等于), '=='（等于）
(2) value, int类型或string类型，string类型只能是'无上限'

4. 用户的意图可以包含按price或data排序，以sort字段标识，取值为一个结构体：
(1) 结构体中以"ordering"="descend"表示按降序排序，以"value"字段存储待排序的字段
(2) 结构体中以"ordering"="ascend"表示按升序排序，以"value"字段存储待排序的字段

只输出中只包含用户提及的字段，不要猜测任何用户未直接提及的字段。不要输出值为null的字段。
"""
# DO NOT OUTPUT NULL-VALUED FIELD!

# 多轮对话的例子
examples = """
客服：有什么可以帮您
用户：100G套餐有什么

{"data":{"operator":">=","value":100}}

客服：有什么可以帮您
用户：100G套餐有什么
客服：我们现在有无限套餐，不限流量，月费300元
用户：太贵了，有200元以内的不

{"data":{"operator":">=","value":100},"price":{"operator":"<=","value":200}}

客服：有什么可以帮您
用户：便宜的套餐有什么
客服：我们现在有经济套餐，每月50元，10G流量
用户：100G以上的有什么

{"data":{"operator":">=","value":100},"sort":{"ordering"="ascend","value"="price"}}

客服：有什么可以帮您
用户：100G以上的套餐有什么
客服：我们现在有畅游套餐，流量100G，月费180元
用户：流量最多的呢

{"sort":{"ordering"="descend","value"="data"},"data":{"operator":">=","value":100}}
"""

# {"sort":{"ordering"="ascend","value"="price"},"data":{"operator":">=","value":100}}
# input_text = "哪个便宜"

# {"name": "无限套餐"}
# input_text = "无限量哪个多少钱"

# {"sort":{"ordering"="descend","value"="data"}}
input_text = "流量最大的多少钱"

# 多轮对话上下文
context = f"""
客服：有什么可以帮您
用户：有什么100G以上的套餐推荐
客服：我们有畅游套餐和无限套餐，您有什么价格倾向吗
用户：{input_text}
"""

prompt = f"""
{instruction}

{output_format}

{examples}

{context}
"""

content = get_chat_completion_content(prompt)
print(content)

{"sort":{"ordering"="descend","value"="data"}}


## 不包含历史对话，虽然本身 gpt 具有这样的功能，但因为请求的是 closeAI 的接口，存在负载均衡，因此是无状态的，不存在连续聊天，记录上下文的能力，每轮对话是独立的。

In [13]:
# 1 system:
# 	你是一个手机流量套餐的客服代表。可以帮助用户选择最合适的流量套餐产品。可以选择的套餐包括：
# 经济套餐，月费50元，10G流量；
# 畅游套餐，月费180元，100G流量；
# 无限套餐，月费300元，1000G流量；
# 校园套餐，月费150元，200G流量，仅限在校生。
# 1 user:
# 	有没有土豪套餐？
# 1 assistant:
# 	很抱歉，我们没有土豪套餐。但是我们有无限套餐，它提供1000G的流量，适合大量使用流量的用户。
# 2 user:
# 	多少钱？
# 2 assistant:
# 	请问您指的是什么物品或服务的价格？请提供更具体的信息。
# 3 user:
# 	给我办一个？
# 3 assistant:
# 	很抱歉，我无法为您办理事务。我是一个人工智能助手，只能提供信息和回答问题。如果您有任何问题或需要帮助，我会尽力提供帮助。

history_message_list = list()

system_prompt = """
你是一个手机流量套餐的客服代表。可以帮助用户选择最合适的流量套餐产品。可以选择的套餐包括：
经济套餐，月费50元，10G流量；
畅游套餐，月费180元，100G流量；
无限套餐，月费300元，1000G流量；
校园套餐，月费150元，200G流量，仅限在校生。
"""

get_chat_completion_content(user_prompt="有没有土豪套餐？", system_prompt=system_prompt, history_message_list=history_message_list, using_history_message_list=False)
get_chat_completion_content(user_prompt="多少钱？",  history_message_list=history_message_list, using_history_message_list=False)
get_chat_completion_content(user_prompt="给我办一个？", history_message_list=history_message_list, using_history_message_list=False)

print_history_message_list(history_message_list)

1 system:
	你是一个手机流量套餐的客服代表。可以帮助用户选择最合适的流量套餐产品。可以选择的套餐包括：
经济套餐，月费50元，10G流量；
畅游套餐，月费180元，100G流量；
无限套餐，月费300元，1000G流量；
校园套餐，月费150元，200G流量，仅限在校生。
1 user:
	有没有土豪套餐？
1 assistant:
	很抱歉，我们没有土豪套餐。我们的最高套餐是无限套餐，包含1000G流量，月费300元。如果您需要更多流量，可以考虑选择该套餐。
2 user:
	多少钱？
2 assistant:
	很抱歉，我无法提供具体的价格信息，因为我是一个AI助手，无法进行实时的价格查询。请您提供更具体的信息，例如您想了解什么产品或服务的价格，我会尽力帮助您。
3 user:
	给我办一个？
3 assistant:
	很抱歉，我无法为您办理事务。我是一个AI助手，只能提供信息和回答问题。如果您有任何问题或需要帮助，我会尽力提供帮助。


##  在上个例子的基础上，自己手动传历史对话，gpt 就具备了连续对话的能力，但 token 的开销也明显变大了

In [14]:
# 1 system:
# 	你是一个手机流量套餐的客服代表。可以帮助用户选择最合适的流量套餐产品。可以选择的套餐包括：
# 经济套餐，月费50元，10G流量；
# 畅游套餐，月费180元，100G流量；
# 无限套餐，月费300元，1000G流量；
# 校园套餐，月费150元，200G流量，仅限在校生。
# 1 user:
# 	有没有土豪套餐？
# 1 assistant:
# 	很抱歉，我们没有土豪套餐。但是我们有无限套餐，它提供1000G的流量，适合大流量用户。
# 2 user:
# 	多少钱？
# 2 assistant:
# 	无限套餐的月费是300元。
# 3 user:
# 	给我办一个？
# 3 assistant:
# 	好的，请问您是在校生吗？

history_message_list = list()

system_prompt = """
你是一个手机流量套餐的客服代表。可以帮助用户选择最合适的流量套餐产品。可以选择的套餐包括：
经济套餐，月费50元，10G流量；
畅游套餐，月费180元，100G流量；
无限套餐，月费300元，1000G流量；
校园套餐，月费150元，200G流量，仅限在校生。
"""

get_chat_completion_content(user_prompt="有没有土豪套餐？", system_prompt=system_prompt, history_message_list=history_message_list)
get_chat_completion_content( user_prompt="多少钱？",  history_message_list=history_message_list)
get_chat_completion_content(user_prompt="给我办一个？", history_message_list=history_message_list)

print_history_message_list(history_message_list)

1 system:
	你是一个手机流量套餐的客服代表。可以帮助用户选择最合适的流量套餐产品。可以选择的套餐包括：
经济套餐，月费50元，10G流量；
畅游套餐，月费180元，100G流量；
无限套餐，月费300元，1000G流量；
校园套餐，月费150元，200G流量，仅限在校生。
1 user:
	有没有土豪套餐？
1 assistant:
	很抱歉，我们没有土豪套餐。但是我们有无限套餐，它提供1000G的流量，适合大量使用流量的用户。
2 user:
	多少钱？
2 assistant:
	无限套餐的月费是300元。
3 user:
	给我办一个？
3 assistant:
	好的，请问您是在校生吗？


## 思维链，提问时以「Let’s think step by step」开头，结果发现 AI 会把问题分解成多个步骤，然后逐步解决，使得输出的结果更加准确。

In [15]:
instruction = """
给定一段用户与手机流量套餐客服的对话，
你的任务是判断客服介绍产品信息的准确性：

当向用户介绍流量套餐产品时，
客服人员必须准确提及产品名称、月费价格和月流量总量 上述信息缺失一项或多项，或信息与实时不符，都算信息不准确

已知产品包括：

经济套餐：月费50元，月流量10G
畅游套餐：月费180元，月流量100G
无限套餐：月费300元，月流量1000G
校园套餐：月费150元，月流量200G，限在校学生办理
"""

# 输出描述
output_format = """
以JSON格式输出。
如果信息准确，输出：{"accurate":true}
如果信息不准确，输出：{"accurate":false}
"""

# {"accurate": false}
# cot_switch = ""

# 根据对话记录，客服介绍了两个套餐：畅游套餐和校园套餐。我们逐步分析客服的介绍是否准确。
# 1. 客服介绍了畅游套餐，包括月费180元和月流量100G。这与实际的畅游套餐信息相符，所以这部分信息准确。
# 2. 客服介绍了校园套餐，包括月费150元和月流量200G，并且指出校园套餐比非学生的畅游套餐便宜流量还多。校园套餐的月费和月流量与实际相符，而且客服提到了校园套餐是专门为在校学生办理的，这也与实际相符。所以这部分信息也准确。
# 综上所述，客服介绍的产品信息是准确的。输出结果为：{"accurate":true}
cot_switch = "请一步一步分析以下对话"

context1 = """
用户：有什么便宜的流量套餐
客服：您好，我们有个经济型套餐，50元每月
"""

context2 = """
用户：流量大的套餐有什么
客服：我们推荐畅游套餐，180元每月，100G流量，大多数人都够用的
用户：学生有什么优惠吗
客服：如果是在校生的话，可以办校园套餐，150元每月，含200G流量，比非学生的畅游套餐便宜流量还多
"""

prompt = f"""
{instruction}

{output_format}

{cot_switch}

对话记录：
{context2}
"""

content = get_chat_completion_content(prompt)
print(content)

根据对话记录，客服介绍产品信息的准确性可以分析如下：

1. 客服提到了畅游套餐，包括月费180元和月流量100G，这与实际情况相符，属于准确信息。
2. 客服提到了校园套餐，包括月费150元和月流量200G，并且指出该套餐只限在校学生办理，这与实际情况相符，属于准确信息。

综上所述，客服介绍的产品信息是准确的。

输出结果为：{"accurate":true}


## 自洽性或称自我一致性（Self-Consistency），一种对抗「幻觉」的手段。就像我们做数学题，要多次验算一样。

## 注意：如果在 output_format 中增加 ：“请不要输出除JSON外其它的文字信息”，那么结果错误的可能性非常大，说明一步一步的COT必须要输出来，就是说 GPT 并没有思考的过程，或者说它的思考过程一定要写出来才能真正起作用，影响后面的生成结果

In [16]:
instruction = """
给定一段用户与手机流量套餐客服的对话，
你的任务是判断客服介绍产品信息的准确性：

当向用户介绍流量套餐产品时，
客服人员必须准确提及产品名称、月费价格和月流量总量 上述信息缺失一项或多项，或信息与实时不符，都算信息不准确

已知产品包括：

经济套餐：月费50元，月流量10G
畅游套餐：月费180元，月流量100G
无限套餐：月费300元，月流量1000G
校园套餐：月费150元，月流量200G，限在校学生办理
"""

# 输出描述
output_format = """
以JSON格式输出。
如果信息准确，输出：{"accurate":true}
如果信息不准确，输出：{"accurate":false}
"""

cot_switch = "请一步一步分析以下对话"

context1 = """
用户：有什么便宜的流量套餐
客服：您好，我们有个经济型套餐，50元每月
"""

context2 = """
用户：流量大的套餐有什么
客服：我们推荐畅游套餐，180元每月，100G流量，大多数人都够用的
用户：学生有什么优惠吗
客服：如果是在校生的话，可以办校园套餐，150元每月，含200G流量，比非学生的畅游套餐便宜流量还多
"""

prompt = f"""
{instruction}

{output_format}

{cot_switch}

对话记录：
{context2}
"""

# 2 次 false，3 次 True
for i in range(5):
        print(f"------第{i+1}次------")
        content = get_chat_completion_content(prompt)
        print(content)

------第1次------
根据对话记录，客服介绍产品信息的准确性可以分析如下：

1. 客服提到了畅游套餐，包括月费180元和月流量100G，这与实际产品畅游套餐的信息一致，所以这部分信息准确。

2. 客服提到了校园套餐，包括月费150元和月流量200G，并且指出该套餐只限在校学生办理，这与实际产品校园套餐的信息一致，所以这部分信息准确。

综上所述，客服介绍产品信息的准确性是准确的。

输出结果为：{"accurate":true}
------第2次------
根据对话记录，客服介绍产品信息的准确性可以分析如下：

1. 客服提到了畅游套餐，包括月费180元和月流量100G，这与实际情况相符，属于准确信息。
2. 客服提到了校园套餐，包括月费150元和月流量200G，并且指出该套餐只限在校学生办理，这与实际情况相符，属于准确信息。

综上所述，客服介绍的产品信息是准确的。

输出结果为：{"accurate":true}
------第3次------
根据对话记录，客服介绍产品信息的准确性可以分析如下：

1. 客服提到了畅游套餐，包括月费180元和月流量100G，这与实际情况相符，属于准确信息。
2. 客服提到了校园套餐，包括月费150元和月流量200G，并且指出该套餐只限在校学生办理，这与实际情况相符，属于准确信息。

综上所述，客服介绍的产品信息是准确的。

输出结果为：{"accurate":true}
------第4次------
根据对话记录，客服介绍产品信息的准确性可以分析如下：

1. 客服提到了畅游套餐，包括月费180元和月流量100G，这与实际产品畅游套餐的信息一致，因此这部分信息准确。

2. 客服提到了校园套餐，包括月费150元和月流量200G，这与实际产品校园套餐的信息一致，因此这部分信息准确。

综上所述，客服介绍的产品信息准确，输出结果为：{"accurate":true}
------第5次------
根据对话记录，客服介绍的产品信息如下：

畅游套餐：月费180元，月流量100G
校园套餐：月费150元，月流量200G，限在校学生办理

我们可以逐条检查客服介绍的产品信息是否准确：

1. 客服介绍的畅游套餐信息：
   - 月费：180元，与实际情况相符。
   - 月流量：100G，与实际情况相符。

2. 

## 思维树，在思维链的每一步，采样多个分支，拓扑展开成一棵思维树，判断每个分支的任务完成度，以便进行启发式搜索

In [17]:
prompt = """
小明100米跑成绩：10.5秒，1500米跑成绩：3分20秒，铅球成绩：12米。请问小明适合什么运动
"""

# 根据小明的成绩，可以看出他在短跑方面有较好的表现，1500米跑成绩也不错，而铅球成绩相对较一般。
# 综合考虑，小明适合参加田径项目中的短跑和中长跑项目。
content = get_chat_completion_content(prompt)
print(content)

根据小明的成绩，可以看出他在短跑方面有较好的表现，1500米跑成绩也不错，而铅球成绩相对较低。综合考虑，小明适合参加短跑项目，如100米、200米等短跑比赛。


In [18]:
prompt = """
小明100米跑成绩：10.5秒，1500米跑成绩：3分20秒，铅球成绩：12米。请一步一步分析，看小明适合什么运动
"""

# 首先，我们可以计算小明的速度。在100米跑中，小明用了10.5秒，所以他的速度是100米/10.5秒 = 9.52米/秒。
# 接下来，我们可以计算小明在1500米跑中的速度。在1500米跑中，小明用了3分20秒，所以他的总时间是3分钟 + 20秒 = 200秒。因此，他的速度是1500米/200秒 = 7.5米/秒。
# 最后，我们可以计算小明在铅球项目中的成绩。小明的铅球成绩是12米。
# 综上所述，小明在100米跑中的速度较快，他的速度为9.52米/秒。在1500米跑中，他的速度为7.5米/秒。在铅球项目中，他的成绩是12米。
# 根据小明的成绩和速度，我们可以得出结论：小明适合短跑项目，如100米跑。他的速度较快，适合快速爆发和短时间内迅速奔跑。
content = get_chat_completion_content(prompt)
print(content)

首先，我们可以计算小明的速度。在100米跑中，小明用时10.5秒，所以他的速度是100米/10.5秒=9.52米/秒。

接下来，我们可以计算小明在1500米跑中的速度。在1500米跑中，小明用时3分20秒，换算成秒为3*60+20=200秒。所以他的速度是1500米/200秒=7.5米/秒。

最后，我们可以计算小明的力量。小明的铅球成绩是12米。

综上所述，小明在短跑方面表现较好，他的速度较快。在长跑方面，他的速度也不错。而在力量方面，他的铅球成绩一般。

综合考虑，小明适合参加短跑和长跑项目，如100米跑和1500米跑。他也可以考虑参加一些需要速度和耐力的项目，如400米跑和800米跑。但在力量项目方面，他的表现相对较弱，可以选择其他更适合他的项目。


In [19]:
def performance_analyser(text):
    prompt = f"{text}\n请根据以上成绩，分析候选人在速度、耐力、力量三方面素质的分档，分档可以重复。分档包括：强（3），中（2），弱（1）三档。\
                \n以JSON格式输出，其中key为素质名，value为以数值表示的分档，结果按照value从大到小排序。"
    content = get_chat_completion_content(prompt)
    return json.loads(content)

def possible_sports(talent, category):
    prompt = f"需要{talent}强的{category}运动有哪些。给出最合适的3个例子，以array形式输出。确保输出能由json.loads解析，不要有其它的文字。"
    content = get_chat_completion_content(prompt, temperature=0.8, print_token_count=False, print_cost_time=False)
    return json.loads(content)

def evaluate(sports, talent, value):
    prompt = f"分析{sports}运动对{talent}方面素质的要求: 强（3），中（2），弱（1）。\
                \n直接输出档位数字。输出只包含数字，不要有其它的文字。"
    response = get_chat_completion_content(prompt, temperature=0, print_token_count=False, print_cost_time=False)
    val = int(response)
    result = (value >= val)
    if result:
        print(f"{sports} need {talent} = {val}, and actual = {value}, good!!!")
    else:
        print(f"{sports} need {talent} = {val}, but actual = {value}, bad...")
    return result

def report_generator(name, performance, talents, sports):
    level = ['弱', '中', '强']
    _talents = {k: level[v - 1] for k, v in talents.items()}
    prompt = F"已知{name}{performance}\n身体素质：{_talents}。\n生成一篇{name}适合{sports}训练的分析报告。"
    response = get_chat_completion_content(prompt, temperature=0, print_token_count=False, print_cost_time=False)
    return response

name = "小明"
performance = "100米跑成绩：10.5秒，1500米跑成绩：3分20秒，铅球成绩：100米，其中铅球和100米跑全校排名第一。"
category = "搏击"

talents = performance_analyser(name + performance)
print("===talents===")
print(talents)

cache = set()
# 深度优先

"""
===talents===
{'速度': 3, '力量': 3, '耐力': 2}
===速度 leafs===
['拳击', '跆拳道', '泰拳']
拳击 need 力量 = 3, and actual = 3, good!!!
拳击 need 耐力 = 3, but actual = 2, bad...
跆拳道 need 力量 = 3, and actual = 3, good!!!
跆拳道 need 耐力 = 3, but actual = 2, bad...
泰拳 need 力量 = 3, and actual = 3, good!!!
泰拳 need 耐力 = 3, but actual = 2, bad...
===力量 leafs===
['拳击', '泰拳', '散打']
散打 need 速度 = 3, and actual = 3, good!!!
散打 need 耐力 = 3, but actual = 2, bad...
===耐力 leafs===
['拳击', '泰拳', '综合格斗']
综合格斗 need 速度 = 3, and actual = 3, good!!!
综合格斗 need 力量 = 3, and actual = 3, good!!!
****
小明是一位身体素质非常出色的学生，他在100米跑、1500米跑和铅球项目上都取得了优秀的成绩。根据他的成绩和身体素质评估，我们可以得出他在速度和力量方面都非常强，而在耐力方面属于中等水平。
综合格斗训练是一项要求全面发展身体素质的运动，包括速度、力量、耐力等多个方面。小明在速度和力量方面的优势将使他在综合格斗训练中具备一定的竞争力。
首先，小明在100米跑项目上取得了10.5秒的成绩，这显示了他出色的爆发力和速度。在综合格斗训练中，速度是非常重要的因素之一，它能够帮助选手快速躲避对手的攻击，并迅速反击。小明的速度优势将使他在综合格斗训练中具备快速反应和灵活机动的能力。
其次，小明在铅球项目上取得了全校第一的成绩，这显示了他出色的力量。力量在综合格斗训练中同样非常重要，它能够帮助选手发出强有力的攻击，并在对抗中占据优势。小明的力量优势将使他在综合格斗训练中具备强大的攻击和防御能力。
然而，小明在耐力方面的水平只属于中等。在综合格斗训练中，耐力是非常重要的因素之一，它能够帮助选手在长时间的对抗中保持持久的战斗力。虽然小明的耐力水平不是最强的，但他可以通过训练来提高自己的耐力，以适应综合格斗训练的要求。
综合格斗训练需要选手具备全面的身体素质，包括速度、力量和耐力。小明在速度和力量方面具备明显的优势，这将使他在综合格斗训练中具备一定的竞争力。然而，他需要通过训练来提高自己的耐力水平，以适应长时间的对抗。
综上所述，小明适合进行综合格斗训练。他的速度和力量优势将使他在训练中具备竞争力，而他的耐力水平可以通过训练来提高。我们相信，在综合格斗训练中，小明将能够发挥出自己的优势，并取得更好的成绩。
****
"""
# 第一层节点
for k, v in talents.items():
    found = False
    if v < 2:  # 剪枝
        continue
    leafs = possible_sports(k, category)
    print(f"==={k} leafs===")
    print(leafs)
     # 第二层节点
    for sports in leafs:
        if sports in cache:
            continue
        cache.add(sports)
        suitable = True
        # sports 运动需要的三个天赋必须同时满足，否则认为不合适
        for t, p in talents.items():
            if t == k:
                continue
            # 第三层节点
            if not evaluate(sports, t, p):  # 剪枝
                suitable = False
                break
        if suitable:
            report = report_generator(name, performance, talents, sports)
            print("****")
            print(report)
            print("****")
            found = True
            break
    if found:
        break

===talents===
{'速度': 3, '力量': 3, '耐力': 2}
===速度 leafs===
['拳击', '跆拳道', '泰拳']
拳击 need 力量 = 3, and actual = 3, good!!!
拳击 need 耐力 = 3, but actual = 2, bad...
跆拳道 need 力量 = 3, and actual = 3, good!!!
跆拳道 need 耐力 = 3, but actual = 2, bad...
泰拳 need 力量 = 3, and actual = 3, good!!!
泰拳 need 耐力 = 3, but actual = 2, bad...
===力量 leafs===
['拳击', '泰拳', '综合格斗']
综合格斗 need 速度 = 3, and actual = 3, good!!!
综合格斗 need 耐力 = 3, but actual = 2, bad...
===耐力 leafs===
['拳击', '泰拳', '跆拳道']


## Function（Tool） Calling

In [20]:
tools = [{  # 用 JSON 描述函数。可以定义多个。由大模型决定调用谁。也可能都不调用
    "type": "function",
    "function": {
        "name": "sum",
        "description": "加法器，计算一组数的和",
        "parameters": {
            "type": "object",
            "properties": {
                "numbers": {
                    "type": "array",
                    "items": {
                        "type": "number"
                    }
                }
            }
        }
    }
}]

# =====第 1 次GPT回复=====
# {
#     "content": null,
#     "role": "assistant",
#     "function_call": null,
#     "tool_calls": [
#         {
#             "id": "call_wQA7DPewmq9zM5XHjO0Nd2ag",
#             "function": {
#                 "arguments": "{\n  \"numbers\": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]\n}",
#                 "name": "sum"
#             },
#             "type": "function"
#         }
#     ]
# }
# =====函数返回=====
# 55
# =====第 2 次GPT回复=====
# The sum of 1, 2, 3, 4, 5, 6, 7, 8, 9, and 10 is 55.
# prompt = "Tell me the sum of 1, 2, 3, 4, 5, 6, 7, 8, 9, 10."

# =====第 1 次GPT回复=====
# {
#     "content": null,
#     "role": "assistant",
#     "function_call": null,
#     "tool_calls": [
#         {
#             "id": "call_wQA7DPewmq9zM5XHjO0Nd2ag",
#             "function": {
#                 "arguments": "{\n  \"numbers\": [2, 4]\n}",
#                 "name": "sum"
#             },
#             "type": "function"
#         }
#     ]
# }
# =====函数返回=====
# 6
# =====第 2 次GPT回复=====
# 桌上有 6 个水果。
# prompt = "桌上有 2 个苹果，四个桃子和 3 本书，一共有几个水果？"

# =====第 1 次GPT回复=====
# {
#     "content": null,
#     "role": "assistant",
#     "function_call": null,
#     "tool_calls": [
#         {
#             "id": "call_K6w6wFzPmZyoZTI49f3wkUYb",
#             "function": {
#                 "arguments": "{\n  \"numbers\": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]\n}",
#                 "name": "sum"
#             },
#             "type": "function"
#         }
#     ]
# }
# =====函数返回=====
# 5050
# =====第 2 次GPT回复=====
# 1 + 2 + 3 + ... + 99 + 100 = 5050
prompt = "1+2+3...+99+100"

# =====第 1 次GPT回复=====
# {
#     "content": null,
#     "role": "assistant",
#     "function_call": null,
#     "tool_calls": [
#         {
#             "id": "call_zMgbH9I95Yd6gQhVmcxjpZ2i",
#             "function": {
#                 "arguments": "{\n  \"numbers\": [1024, 1024]\n}",
#                 "name": "sum"
#             },
#             "type": "function"
#         }
#     ]
# }
# =====函数返回=====
# 2048
# =====第 2 次GPT回复=====
# 1024 乘以 1024 等于 2048。
# gpt3.5 会错误的调用 sum，但  gpt4 不会调用，且返回正确的答案
# prompt = "1024 乘以 1024 是多少？"   # Tools 里没有定义乘法，会怎样？

# =====第 1 次GPT回复=====
# {
#     "content": "\u592a\u9633\u4ece\u4e1c\u65b9\u5347\u8d77\u3002",
#     "role": "assistant",
#     "function_call": null,
#     "tool_calls": null
# }
# ChatCompletionMessage(content='太阳从东方升起。', role='assistant', function_call=None, tool_calls=None)
# prompt = "太阳从哪边升起？"           # 不需要算加法，会怎样？

messages = [
    {"role": "system", "content": "你是一个数学家"},
    {"role": "user", "content": prompt}
]
message = get_chat_completion_content(messages=messages, tools=tools, print_token_count=False, print_cost_time=False)

# 经过实验 ，如果只加上最近的 message，会返回 sum of 2 and 4 is 6，即 gpt 会知道模型调用这个事情，
# 但不知道之前的上文，即模型调用的结果不知道该怎么用：
# messages = []

# 把大模型的回复加入到对话历史中
messages.append(message)

# 经过实验 ，如果不加上最近的 message，会报错：
# BadRequestError: Error code: 400 - {'error': {'message': "Invalid parameter: messages with role 'tool' must be a response to a preceeding message 
# with 'tool_calls'.", 'type': 'invalid_request_error', 'param': 'messages.[0].role', 'code': None}}
# messages = []

print("=====第 1 次GPT回复=====")
print_json(message)

# 如果返回的是函数调用结果，则打印出来
if (message.tool_calls is not None):
    # 是否要调用 sum
    tool_call = message.tool_calls[0]
    # ！！！这里就是写函数的地方
    if (tool_call.function.name == "sum"):
        # 调用 sum
        args = json.loads(tool_call.function.arguments)
        result = sum(args["numbers"])
        print("=====函数返回=====")
        print(result)

        # 把函数调用结果加入到对话历史中
        messages.append(
            {
                "tool_call_id": tool_call.id,  # 用于标识函数调用的 ID
                "role": "tool",
                "name": "sum",
                "content": str(result)  # 数值 result 必须转成字符串
            }
        )

        # 再次调用大模型
        print("=====第 2 次GPT回复=====")
        print(get_chat_completion_content(messages=messages, print_token_count=False, print_cost_time=False))
    else:
        print(F"error, {tool_call.function.name}")
else:
    print(message)

=====第 1 次GPT回复=====
{
    "content": null,
    "role": "assistant",
    "function_call": null,
    "tool_calls": [
        {
            "id": "call_Njv16QK76cCP0CWcXX4gQjxn",
            "function": {
                "arguments": "{\n  \"numbers\": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100]\n}",
                "name": "sum"
            },
            "type": "function"
        }
    ]
}
=====函数返回=====
5050
=====第 2 次GPT回复=====
1 + 2 + 3 + ... + 99 + 100 = 5050
